In [ ]:
# lib
import sys

sys.path.append("..")

# set cwd one up
import os

os.chdir("..")
path = "data/hes/Hes1_example.csv"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
import gpflow

print(gpflow.config.default_positive_bijector())
print(gpflow.config.default_positive_minimum())
# gpflow.config.set_default_positive_bijector("exp")
# print(gpflow.config.default_positive_bijector())

In [ ]:
from gpcell import OscillatorDetector
from gpcell.backend.priors import (
    sd_ou_priors,
    sd_ouosc_priors,
)

In [ ]:
params = {
    "plots": ["MCMC", "MCMC_marginal"],
    "verbose": True,
    "joblib": True,
    "ou_prior_gen": sd_ou_priors,
    "ouosc_prior_gen": sd_ouosc_priors,
}
od: OscillatorDetector = OscillatorDetector.from_file(
    path, "Time (h)", "Background", "Cell", params
)
od.fit(methods="MCMC")


In [ ]:
od.mcmc_plot

In [ ]:
od.marginal_plot

# MCMC Dev

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.optimize import fixed_point

import pymc as pm
from pymc.gp.cov import ExpQuad, Cosine

from gpcell.backend.priors import hes_mcmc_ou_priors, hes_mcmc_ouosc_priors


def bridge_log_marginal(log_post, log_q, tol=1e-6, maxiter=50):
    """
    Fixed-point solver for log Z in the (optimal) bridge sampling identity.
    """

    def fp(logm):
        w = np.exp(log_post - log_q - logm)
        return logm - np.log(np.mean(1.0 / w))

    return fixed_point(fp, 0.0, xtol=tol, maxiter=maxiter)


def marginal_logZ(model_name, X, y, noise_sd, draws=1000, tune=1000):
    """
    Build & sample a tiny PyMC GP Marginal model for either 'ou' or 'ouosc',
    then estimate logZ by bridge sampling on the MCMC draws.
    """
    # 1) pull out the TFP priors and their bounds
    if model_name == "ou":
        prior = hes_mcmc_ou_priors(noise_sd)
        ls_key = "kernel.lengthscales.prior"
        amp_key = "kernel.variance.prior"
        has_second = False
    else:
        prior = hes_mcmc_ouosc_priors(noise_sd)
        ls_key = "kernel.kernels[0].lengthscales.prior"
        amp_key = "kernel.kernels[0].variance.prior"
        ls2_key = "kernel.kernels[1].lengthscales.prior"
        has_second = True

    low_ls = float(prior[ls_key].low.numpy())
    high_ls = float(prior[ls_key].high.numpy())
    low_amp = float(prior[amp_key].low.numpy())
    high_amp = float(prior[amp_key].high.numpy())
    if has_second:
        low_ls2 = float(prior[ls2_key].low.numpy())
        high_ls2 = float(prior[ls2_key].high.numpy())

    # 2) build & sample the PyMC3 GP
    with pm.Model() as pm_model:
        print(f"values: {low_ls}, {high_ls}, {low_amp}, {high_amp}")
        ls = pm.Uniform("ls", lower=low_ls, upper=high_ls)
        amp = pm.Uniform("amp", lower=low_amp, upper=high_amp)

        if not has_second:
            cov = amp**2 * ExpQuad(1, ls=ls)
        else:
            ls2 = pm.Uniform("ls2", lower=low_ls2, upper=high_ls2)
            cov = amp**2 * ExpQuad(1, ls=ls) * Cosine(1, ls=ls2)

        gp = pm.gp.Marginal(cov_func=cov)
        gp.marginal_likelihood("y_obs", X=X, y=y, noise=noise_sd)

        trace = pm.sample(
            draws=draws,
            tune=tune,
            chains=2,
            target_accept=0.9,
            return_inferencedata=False,
        )

    # 3) assemble the ASCII‐only variable list
    var_list = ["ls", "amp"]
    if has_second:
        var_list.append("ls2")

    # 4) extract draws into an (N × D) array
    samples = np.column_stack([trace[v] for v in var_list])

    # 5) fit a Gaussian proposal q(θ) to the posterior draws
    prop = multivariate_normal(mean=samples.mean(axis=0), cov=np.cov(samples.T))

    # 6) compile the model's logp function, passing the PyMC RVs themselves
    #    rather than strings, so the right transformed names get wired up.
    rv_objs = [pm_model.named_vars[name] for name in var_list]
    logp_fn = pm_model.compile_logp(vars=rv_objs)

    # 7) evaluate the unnormalized posterior on each draw
    points = [dict(zip(var_list, theta)) for theta in samples]
    log_post = np.array([logp_fn(pt) for pt in points])

    # 8) evaluate log q(θ) under our Gaussian proposal
    log_q = prop.logpdf(samples)

    # 9) solve for logZ
    return bridge_log_marginal(log_post, log_q)


In [ ]:
params = {
    "plots": ["MCMC", "MCMC_marginal"],
    "verbose": True,
    "joblib": True,
    "ou_prior_gen": hes_mcmc_ou_priors,
    "ouosc_prior_gen": hes_mcmc_ouosc_priors,
}
od: OscillatorDetector = OscillatorDetector.from_file(
    path, "Time (h)", "Background", "Cell", params
)
od.fit(methods="MCMC")
od.mcmc_plot

In [ ]:
# Now for each cell:
for i, (X_i, Y_i, noise) in enumerate(
    zip(od.X, od.Y_detrended, od.noise_list), start=1
):
    y_flat = Y_i.flatten()
    logZ_ou = marginal_logZ("ou", X_i, y_flat, noise)
    logZ_ouosc = marginal_logZ("ouosc", X_i, y_flat, noise)
    BF = np.exp(logZ_ouosc - logZ_ou)
    print(
        f"Cell {i:02d}: logZ_ou={logZ_ou:.2f}, logZ_ouosc={logZ_ouosc:.2f}, BF={BF:.1f}"
    )


In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.optimize import fixed_point

import pymc as pm
from pymc.gp.cov import Cosine

from gpcell.backend.priors import hes_mcmc_ou_priors, hes_mcmc_ouosc_priors


def bridge_log_marginal(log_post, log_q, tol=1e-6, maxiter=50):
    """
    Fixed-point solver for log Z in the (optimal) bridge sampling identity.
    """

    def fp(logm):
        w = np.exp(log_post - log_q - logm)
        return logm - np.log(np.mean(1.0 / w))

    return fixed_point(fp, 0.0, xtol=tol, maxiter=maxiter)


def marginal_logZ(model_name, X, y, noise_sd, draws=1000, tune=1000):
    """
    Build & sample a small PyMC3 GP Marginal model for either 'ou' or 'ouosc',
    then estimate logZ by bridge sampling on the MCMC draws.
    """
    # 1) Extract your TFP priors and pull out their low/high for Uniform
    if model_name == "ou":
        prior = hes_mcmc_ou_priors(noise_sd)
        low_ls = float(prior["kernel.lengthscales.prior"].low.numpy())
        high_ls = float(prior["kernel.lengthscales.prior"].high.numpy())
        low_ev = float(prior["kernel.variance.prior"].low.numpy())
        high_ev = float(prior["kernel.variance.prior"].high.numpy())
        varnames = ["ℓ", "η"]
    else:  # "ouosc"
        prior = hes_mcmc_ouosc_priors(noise_sd)
        low_ls = float(prior["kernel.kernels[0].lengthscales.prior"].low.numpy())
        high_ls = float(prior["kernel.kernels[0].lengthscales.prior"].high.numpy())
        low_ev = float(prior["kernel.kernels[0].variance.prior"].low.numpy())
        high_ev = float(prior["kernel.kernels[0].variance.prior"].high.numpy())
        low_ls2 = float(prior["kernel.kernels[1].lengthscales.prior"].low.numpy())
        high_ls2 = float(prior["kernel.kernels[1].lengthscales.prior"].high.numpy())
        varnames = ["ℓ1", "η1", "ℓ2"]

    # 2) Build & sample the small PyMC3 GP
    with pm.Model() as pm_model:
        # common OU hyperpriors
        ℓ = pm.Uniform("ℓ", lower=low_ls, upper=high_ls)
        η = pm.Uniform("η", lower=low_ev, upper=high_ev)

        if model_name == "ou":
            cov = η**2 * pm.gp.cov.ExpQuad(1, ls=ℓ)
        else:
            ℓ2 = pm.Uniform("ℓ2", lower=low_ls2, upper=high_ls2)

            cov = η**2 * pm.gp.cov.ExpQuad(1, ls=ℓ) * Cosine(1, ls=ℓ2)

        gp = pm.gp.Marginal(cov_func=cov)
        gp.marginal_likelihood("y_obs", X=X, y=y, noise=noise_sd)

        trace = pm.sample(
            draws=draws,
            tune=tune,
            chains=2,
            target_accept=0.9,
            return_inferencedata=False,
        )

    # 3) Extract posterior draws into an (N × D) matrix
    samples = np.column_stack([trace[v] for v in varnames])

    # 4) Fit a Gaussian proposal q(θ) to those draws
    prop = multivariate_normal(mean=samples.mean(axis=0), cov=np.cov(samples.T))

    # 5) Compile model
    logp_fn = pm_model.compile_logp(vars=[ℓ, η])

    # 6) Evaluate on all draws
    points = [dict(zip(varnames, θ)) for θ in samples]
    log_post = np.array([logp_fn(pt) for pt in points])

    # 7) Build proposal & bridge-sample as before
    prop = multivariate_normal(samples.mean(axis=0), np.cov(samples.T))
    log_q = prop.logpdf(samples)

    # 8) Solve for logZ
    logZ = bridge_log_marginal(log_post, log_q)
    return logZ


# Loop over cells, compute bridge-sampled logZ for each model, then BF
for i, (X_i, Y_i, noise) in enumerate(zip(od.X, od.Y_detrended, od.noise_list)):
    y_flat = Y_i.flatten()
    logZ_ou = marginal_logZ("ou", X_i, y_flat, noise)
    logZ_ouosc = marginal_logZ("ouosc", X_i, y_flat, noise)
    BF = np.exp(logZ_ouosc - logZ_ou)
    print(
        f"Cell {i + 1:02d}:  logZ_ou = {logZ_ou:.2f},  logZ_ouosc = {logZ_ouosc:.2f},  BF = {BF:.1f}"
    )
